In [5]:
import pandas as pd
import regex as re
import string
import json
import csv

### Document ID and Answer Text

In [12]:
collection = pd.read_csv("train/FiQA_train_doc_final.tsv", sep="\t")
collection = collection[['docid', 'doc']]
collection = collection.sort_values(by=['docid'])

collection.head(5)

,docid,doc
0,3,I'm not saying I don't like the idea of on-the...
1,31,So nothing preventing false ratings besides ad...
2,56,You can never use a health FSA for individual ...
3,59,Samsung created the LCD and other flat screen ...
4,63,Here are the SEC requirements: The federal sec...


In [15]:
print("Example passage: \n")
print(collection.iloc[0]['doc'])

Example passage: 

I'm not saying I don't like the idea of on-the-job training too, but you can't expect the company to do that. Training workers is not their job - they're building software. Perhaps educational systems in the U.S. (or their students) should worry a little about getting marketable skills in exchange for their massive investment in education, rather than getting out with thousands in student debt and then complaining that they aren't qualified to do anything.


In [7]:
print("Total number of passages: {}".format(len(collection)))

Total number of passages: 57638


In [85]:
# Write collection df to file
with open("fiqa-passage/collection.tsv",'w') as write_tsv:
    write_tsv.write(collection.to_csv(sep='\t', index=False, header=False))

In [86]:
# Convert collection df to JSON file for Anserini's document indexer
output_jsonl_file = open("fiqa-passage/collection_jsonl/docs00.json", 'w', encoding='utf-8', newline='\n')

with open("fiqa-passage/collection.tsv", encoding='utf-8') as f:
    for i, line in enumerate(f):
        id_text = line.rstrip().split('\t')
        doc_id, doc_text = id_text[0], id_text[1]
        
        output_dict = {'id': doc_id, 'contents': doc_text}
        
        output_jsonl_file.write(json.dumps(output_dict) + '\n')

### Question ID and Question Text

In [18]:
# Question ID and Question text pair
query_df = pd.read_csv("train/FiQA_train_question_final.tsv", sep="\t")

queries = query_df[['qid', 'question']]

queries.head(5)

,qid,question
0,0,What is considered a business expense on a bus...
1,1,Claiming business expenses for a business with...
2,2,Transferring money from One business checking ...
3,3,Having a separate bank account for business/in...
4,4,Business Expense - Car Insurance Deductible Fo...


In [19]:
print("Example Question: \n")
print(queries.iloc[0]['question'])

Example Question: 

What is considered a business expense on a business trip?


In [22]:
print("Number of questions: {}".format(len(queries)))

Number of questions: 6648


In [ ]:
# Write queries to file
with open("fiqa-passage/queries_train.tsv",'w') as write_tsv:
    write_tsv.write(queries.to_csv(sep='\t', index=False, header=False))

### Question ID and Answer ID pair

In [24]:
# Question ID and Answer ID pair
qid_docid = pd.read_csv("train/FiQA_train_question_doc_final.tsv", sep="\t")

qid_docid = qid_docid [['qid', 'docid']]

qid_docid.head(5)
# qid_docid = qid_docid.assign(new1=0)
# qid_docid = qid_docid.assign(new2=1)
# qrels_train = qid_docid[['qid', 'new1', 'docid', 'new2']]

,qid,docid
0,0,18850
1,1,14255
2,2,308938
3,3,296717
4,3,100764


In [30]:
print("Number of Question Answer Pairs: {} \n".format(len(qid_docid)))

Number of Question Answer Pairs: 17110 



In [31]:
# with open("fiqa-passage/qrels_train.tsv",'w') as write_tsv:
#     write_tsv.write(qrels_train.to_csv(sep='\t', index=False, header=False))
# rel = qid_docid['qid'] == 0

# rel_df = qid_docid[rel]

### Example QA pair

In [39]:
print(queries.at[3, 'question'])

x = collection[collection['docid']==296717]

print()
print(x.at[28732, 'doc'])

Having a separate bank account for business/investing, but not a “business account?”

Having a separate checking account for the business makes sense. It simplifies documenting your income/expenses. You can "explain" every dollar entering and exiting the account without having to remember that some of them were for non-business items. My credit union allowed me to have a 2nd checking account and allowed me to put whatever I wanted as the name on the check. I think this looked a little better than having my name on the check. I don't see the need for a separate checking account for investing. The money can be kept in a separate savings account that has no fees, and can even earn a little interest. Unless you are doing a lot of investment transactions a month this has worked for me. I fund IRAs and 529 plans this way. We get paychecks 4-5 times a month, but send money to each of the funds once a month. You will need a business account if the number of transactions becomes large. If you d